## Image segmentation of the digits:

In [24]:
# Packages required:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import cv2
import glob
import os

### Function to segment the image :
In this snippet the path of the directory where the image segments are to be stored is passed along with the count, which is gives the stored image a distinct file_name.

In [25]:
# Function to segment the image:
def segmentImage(image, path, count):

    #Resizing the image and converting to grayscale:
    res = cv2.resize(image ,None,fx=5, fy=5, interpolation = cv2.INTER_CUBIC)

    #Conversion to grayscale:
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    #plt.imshow(gray, 'gray')

    # Filtering out the noise using median filtering:
    blur = cv2.medianBlur(gray, 15)
    #plt.imshow(blur, 'gray')

    some = np.percentile(gray, 80)
    #print some

    # Binarization :
    ret, th2 = cv2.threshold(blur,some,255,cv2.THRESH_BINARY)
    #plt.imshow(th2, 'gray')

    # Connected component analysis : (CCA)
    im = np.uint8(th2)
    _, markers = cv2.connectedComponents(im)

    # Iterate through, segment out the image of each number:
    for i in range(1, np.max(markers) + 1):
        x = (markers == i)
        y = np.sum(x, axis = 0)
        z = np.sum(x, axis = 1)
        max_x = 0
        min_x = 0
        # Finding the max width of a number(component) :
        for j in range(y.shape[0]):
            if y[j] != 0:
                max_x = j
        for j in range(y.shape[0]):
            if y[j] != 0:
                break
            min_x = j

        # Final setting of the dimensions of each number image:
        min_x = min_x - 10
        max_x = max_x + 10
        min_y = 0
        max_y = z.shape[0]
        
        #Writing the required image as a file :
        b = str(count)
        file_name = 'im'+b+'.jpg'
        cv2.imwrite(os.path.join(path, file_name),res[min_y:max_y, min_x:max_x])
        
        #incrementing the image file counter:
        count = count + 1
    return count



### Driver Function :
Here path input is the directory in which the images after the extraction of the required number in the detol bottle are stored.

In [26]:
# Main Driver Function:
# Path to directory for storing the segmented images :
path_output = '/home/ignitarium/Desktop/Text_seg/segmentedNum'

# Path to directory with all the images :
path_input = '/home/ignitarium/Desktop/Text_seg/frames/'
images = [cv2.imread(file) for file in glob.glob(path_input+'*.JPG')]

# Counter for new image file names:
count = 0

#calling segmenting function :
for img in images:
    count = segmentImage(img, path_output, count)